In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import os
import time
import re
import json
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from pearlsim.ml_utilities import *
import pickle
import openmc
import openmc.deplete
from copy import deepcopy
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential, clone_model, load_model
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_validate
from itertools import product
import tensorflow as tf
from scipy.interpolate import PchipInterpolator

2024-06-19 16:35:22.110081: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 16:35:22.456473: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-19 16:35:22.461143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 16:35:30.482669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load Data
The data used in this notebook is created in the generate_kernal_burnup_data notebook. It requires running a lot of Serpent calculations, hence why I made it a separate notebook. First we load and examine the data.

The input features to this model include:
- Depletion Time: the time in days that the pebble has been exposed to flux
- Surface current: Labeled by energy bin, the number of incoming neutrons at a certain energy group imposing on the surface of the pebble per second
- Power: The power produced by the pebble in Watts
- Starting isotope concentrations: The amount of each nuclide present in the pebble in atom density. 

The output features to this model include:
- Ending isotope concentrations: The amount of each nuclide present in the pebble AFTER a burnup step.

Note that the nuclide concentrations and surface current values can be quite large and span orders of magnitude.

In [2]:
raw_features = pd.read_csv("training_data/burnup_features.csv", index_col=0)
flux_targets = pd.read_csv("training_data/burnup_flux.csv", index_col=0).drop(columns=["total_pebble_flux"])
xs_targets = pd.read_csv("training_data/burnup_xs.csv", index_col=0)
raw_burnup_targets = pd.read_csv("training_data/burnup_final_concentrations.csv", index_col=0)

In [3]:

current_features = ['current_E1', 'current_E2','current_E3','current_E4','current_E5','current_E6',
            'current_E7', 'current_E8', 'current_E9', 'current_E10',
            'current_E11', 'current_E12', 'current_E13', 'current_E14',
            'current_E15', 'current_E16','current_E17','current_E18']

ZAI_list = [922350, 922380, 942390, 942410, 541350, 551370, 10010, 10030, 20040, 30070, 40100, 50100, 50110, 60120, 60140, 80160, 80170, 290670, 300660, 300670, 300680, 300700, 300720, 310690, 310710, 310720, 320720, 320730, 320740, 320760, 320770, 320780, 330750, 330760, 330770, 330780, 330790, 330810, 340760, 340770, 340780, 340790, 340791, 340800, 340810, 340811, 340820, 340830, 340840, 340850, 340860, 350790, 350810, 350820, 350830, 350840, 350841, 350850, 350860, 350870, 350880, 360800, 360810, 360820, 360830, 360831, 360840, 360850, 360851, 360860, 360870, 360880, 360890, 360900, 360910, 370830, 370840, 370850, 370860, 370870, 370880, 370890, 370900, 370901, 370910, 370920, 370930, 380860, 380870, 380880, 380890, 380900, 380910, 380920, 380930, 380940, 380950, 390880, 390890, 390891, 390900, 390910, 390911, 390920, 390930, 390940, 390950, 390960, 390961, 390970, 400900, 400910, 400920, 400930, 400940, 400950, 400960, 400970, 400980, 400990, 401000, 401010, 401020, 410930, 410931, 410940, 410950, 410951, 410960, 410970, 410980, 410981, 410990, 410991, 411000, 411010, 420940, 420950, 420960, 420970, 420980, 420990, 421000, 421010, 421020, 421030, 421040, 421050, 421060, 430980, 430990, 430991, 431000, 431010, 431020, 431030, 431040, 431050, 431060, 431070, 440990, 441000, 441010, 441020, 441030, 441040, 441050, 441060, 441070, 441080, 441090, 451020, 451021, 451030, 451031, 451040, 451041, 451050, 451051, 451060, 451061, 451070, 451080, 451090, 461040, 461050, 461060, 461070, 461080, 461090, 461100, 461110, 461120, 471090, 471091, 471101, 471110, 471111, 471120, 471130, 471150, 481100, 481110, 481120, 481130, 481131, 481140, 481150, 481151, 481160, 481170, 481171, 481180, 491130, 491150, 491151, 491170, 491171, 491191, 501150, 501160, 501170, 501171, 501180, 501190, 501191, 501200, 501210, 501211, 501220, 501230, 501231, 501240, 501250, 501251, 501260, 501270, 501271, 501280, 501290, 501291, 501300, 501301, 501310, 501311, 501320, 511210, 511220, 511230, 511240, 511250, 511260, 511261, 511270, 511280, 511281, 511290, 511300, 511301, 511310, 511320, 511321, 511330, 521220, 521230, 521231, 521240, 521250, 521251, 521260, 521270, 521271, 521280, 521290, 521291, 521300, 521310, 521311, 521320, 521330, 521331, 521340, 521350, 521360, 531260, 531270, 531280, 531290, 531300, 531301, 531310, 531320, 531321, 531330, 531340, 531341, 531350, 531360, 531361, 531370, 531380, 541280, 541290, 541300, 541310, 541311, 541320, 541330, 541331, 541340, 541351, 541360, 541370, 541380, 541390, 541400, 551320, 551330, 551340, 551341, 551350, 551351, 551360, 551380, 551381, 551390, 551400, 551410, 561320, 561340, 561350, 561360, 561370, 561371, 561380, 561390, 561400, 561410, 561420, 561430, 561440, 561450, 571370, 571380, 571390, 571400, 571410, 571420, 571430, 571440, 571450, 571460, 571461, 581380, 581390, 581400, 581410, 581420, 581430, 581440, 581450, 581460, 581470, 581480, 591410, 591420, 591421, 591430, 591440, 591441, 591450, 591460, 591470, 591480, 591481, 591490, 591510, 601420, 601430, 601440, 601450, 601460, 601470, 601480, 601490, 601500, 601510, 601520, 601530, 611460, 611470, 611480, 611481, 611490, 611510, 611520, 611530, 611540, 621470, 621480, 621490, 621500, 621510, 621520, 621530, 621540, 621550, 621560, 621570, 621580, 631510, 631520, 631530, 631540, 631541, 631550, 631560, 631570, 631580, 631590, 641520, 641540, 641550, 641560, 641570, 641580, 641590, 641600, 651580, 651590, 651600, 651610, 661600, 661610, 661620, 661630, 661640, 661660, 671650, 671660, 671661, 681660, 681670, 681680, 681690, 681700, 691690, 691710, 701720, 902310, 902320, 902340, 912310, 912340, 922320, 922340, 922360, 922370, 922390, 932370, 932380, 932390, 942380, 942400, 942420, 952410]

def rename_ZAI_columns(dataframe_columns):
    '''
    Convert the isotope headers of a dataframe from ZAId integer format into symbol-mass string format.
    '''
    z_map = {'Ac': 89, 'Ag': 47, 'Al': 13, 'Am': 95, 'Ar': 18, 'As': 33, 'At': 85, 'Au': 79, 'B': 5, 'Ba': 56, 'Be': 4,
             'Bh': 107, 'Bi': 83, 'Bk': 97, 'Br': 35, 'C': 6, 'Ca': 20, 'Cd': 48, 'Ce': 58,
             'Cf': 98, 'Cl': 17, 'Cm': 96, 'Co': 27, 'Cr': 24, 'Cs': 55, 'Cu': 29, 'Ds': 110, 'Db': 105, 'Dy': 66, 'Er': 68,
             'Es': 99, 'Eu': 63, 'F': 9, 'Fe': 26, 'Fm': 100, 'Fr': 87, 'Ga': 31, 'Gd':
                 64, 'Ge': 32, 'H': 1, 'He': 2, 'Hf': 72, 'Hg': 80, 'Ho': 67, 'Hs': 108, 'I': 53, 'In': 49, 'Ir': 77,
             'K': 19, 'Kr': 36, 'La': 57, 'Li': 3, 'Lr': 103, 'Lu': 71, 'Md': 101, 'Mg': 12, 'Mn':
                 25, 'Mo': 42, 'Mt': 109, 'N': 7, 'Na': 11, 'Nb': 41, 'Nd': 60, 'Ne': 10, 'Ni': 28, 'No': 102, 'Np': 93,
             'O': 8, 'Os': 76, 'P': 15, 'Pa': 91, 'Pb': 82, 'Pd': 46, 'Pm': 61, 'Po': 84, 'Pr':
                 59, 'Pt': 78, 'Pu': 94, 'Ra': 88, 'Rb': 37, 'Re': 75, 'Rf': 104, 'Rg': 111, 'Rh': 45, 'Rn': 86, 'Ru': 44,
             'S': 16, 'Sb': 51, 'Sc': 21, 'Se': 34, 'Sg': 106, 'Si': 14, 'Sm': 62, 'Sn': 50,
             'Sr': 38, 'Ta': 73, 'Tb': 65, 'Tc': 43, 'Te': 52, 'Th': 90, 'Ti': 22, 'Tl': 81, 'Tm': 69, 'U': 92, 'V': 23,
             'W': 74, 'Xe': 54, 'Y': 39, 'Yb': 70, 'Zn': 30, 'Zr': 40}
    z_map_inv = {v: k for k, v in z_map.items()}
    rename_map = {}
    particle_counts = {}

    for raw_column in dataframe_columns:
        if "<lib>" in str(raw_column):
            column = raw_column.replace("<lib>","0")
        else:
            column = str(raw_column)
        iso_num = int(column[-1])
        
        if len(column) == 5:
            z = int(column[0])
            a = int(column[1:4])
        else:
            z = int(column[0:2])
            a = int(column[2:5])
        if a > 309:
            iso_num += 1
            a -= 100
        renamed = f"{z_map_inv[z]}{a}"

        if iso_num > 0:
            renamed += f"_m{iso_num}"
        rename_map[raw_column] = renamed
        particle_counts[renamed] = (z,a)
    #renamed_dataframe = dataframe.rename(columns=rename_map)

    return rename_map, particle_counts

non_concentration_columns = current_features+["power", "temperature", "depletion_time"]
raw_nuclide_headers = list(raw_burnup_targets)#.drop(columns=drop_columns).columns)
rename_map,particle_counts = rename_ZAI_columns(raw_nuclide_headers)

missing_nuclides = ["Gd153_m1", "Ag210_m1", "Pm248_m1"]
features = raw_features.rename(columns=rename_map).drop(columns=missing_nuclides)
burnup_targets = raw_burnup_targets.rename(columns=rename_map).drop(columns=missing_nuclides)

In [5]:
with open("pearlsim/rename_map_var.py","w") as f:
    f.write(json.dumps(rename_map))

In [4]:
display(features.iloc[0:3])
display(flux_targets.iloc[0:3])
display(xs_targets.iloc[0:3])
display(burnup_targets.iloc[0:3])

,H3,C12,C13,O16,Se74,Se80,Se82,Br81,Kr80,Kr82,...,current_E12,current_E13,current_E14,current_E15,current_E16,current_E17,current_E18,power,depletion_time,temperature
0,1.588984e-14,0.011691,0.000126,0.035463,1.247539e-14,4.080364e-06,1.032088e-05,6.574997e-06,1.474235e-11,1.474651e-07,...,1.918580e+15,2.767560e+14,1.337520e+15,6.751830e+14,2.986080e+14,5.010360e+14,1.946540e+14,1336.650,6.525,959.0
1,2.313322e-17,0.011691,0.000125,0.035463,1.333009e-17,1.774031e-07,4.467850e-07,2.803852e-07,2.015239e-13,1.353061e-10,...,8.995880e+13,1.326100e+13,6.275400e+13,3.220640e+13,1.420010e+13,2.883840e+13,1.073100e+13,390.635,6.525,959.0
2,5.551811e-20,0.011691,0.000125,0.035463,1.917643e-18,7.524731e-08,1.895287e-07,1.187727e-07,8.039342e-14,3.714896e-11,...,8.588680e+13,1.227590e+13,6.161790e+13,2.955530e+13,1.424780e+13,2.539830e+13,9.203530e+12,442.500,1.631,959.0


,2.50000E-09,7.50000E-09,1.25000E-08,1.75000E-08,2.25000E-08,2.75000E-08,3.25000E-08,3.85000E-08,4.60000E-08,5.40000E-08,...,8.92150E-02,1.47000E-01,2.42750E-01,4.01250E-01,6.60500E-01,1.08700E+00,1.79200E+00,2.95500E+00,4.87225E+00,5.00000E+36
0,5.095980e+11,7.817140e+11,8.506140e+11,9.550200e+11,1.015120e+12,1.043990e+12,1.092850e+12,1.561800e+12,1.875540e+12,1.913320e+12,...,6.135060e+12,6.145600e+12,6.446900e+12,7.544280e+12,8.359570e+12,7.731550e+12,6.097980e+12,3.819860e+12,2.061220e+12,9.460030e+11
1,7.939600e+10,1.268420e+11,1.408100e+11,1.580860e+11,1.701370e+11,1.761820e+11,1.837730e+11,2.636840e+11,3.173770e+11,3.255040e+11,...,2.895200e+11,2.979330e+11,3.319670e+11,4.163210e+11,5.102800e+11,5.848730e+11,5.637180e+11,4.095200e+11,2.123600e+11,7.478940e+10
2,8.263500e+10,1.317260e+11,1.460110e+11,1.638170e+11,1.752720e+11,1.813670e+11,1.900930e+11,2.722260e+11,3.276520e+11,3.352810e+11,...,2.747080e+11,2.804760e+11,3.135640e+11,4.072320e+11,5.116300e+11,5.600050e+11,5.430820e+11,4.017500e+11,2.082250e+11,6.819920e+10


,10030-16-0,60120-102-0,60120-103-0,60120-107-0,80160-16-0,80160-102-0,80160-103-0,80160-107-0,340740-16-0,340740-102-0,...,962490-16-0,962490-17-0,962490-18-1,962490-102-0,962500-16-0,962500-17-0,962500-18-1,962500-102-0,962500-18-3,962500-18-2
0,2.227620e-08,6.200750e-07,5.919920e-11,0.000002,1.504210e-14,3.151560e-08,4.691630e-09,0.000007,8.847440e-09,0.230994,...,0.000016,1.638290e-09,0.051779,0.023426,0.000004,9.488890e-09,0.000035,0.124673,0.000183,0.001131
1,3.290960e-08,6.891490e-07,2.097660e-10,0.000003,4.204440e-12,3.456860e-08,2.313130e-08,0.000012,3.327730e-08,0.203606,...,0.000027,6.266240e-09,0.047565,0.020551,0.000006,3.947770e-08,0.000032,0.111969,0.000347,0.002027
2,3.168710e-08,7.314250e-07,3.052960e-10,0.000003,8.620530e-12,3.699870e-08,2.614510e-08,0.000012,4.204020e-08,0.229704,...,0.000025,7.689630e-09,0.050428,0.022222,0.000006,4.516190e-08,0.000034,0.122559,0.000336,0.001977


,H1,H2,H3,He3,He4,Be9,B11,B12,C12,C13,...,I122,Cs128,Cd104,In110_m1,Sn110,Xe122,Sm142,Tc94,La134,Rn216
0,9.465268e-10,9.436275e-11,1.695195e-14,2.302246e-15,7.261703e-07,1.466388e-08,2.912315e-14,1.602450e-21,0.011691,0.000126,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.635691e-11,7.361993e-13,4.808605e-17,1.389541e-18,8.433346e-09,4.705992e-10,5.811842e-15,2.322536e-22,0.011691,0.000125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.762000e-13,5.575177e-14,7.771446e-18,7.396430e-21,2.206188e-09,2.434211e-10,2.559732e-15,3.913864e-22,0.011691,0.000125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Standardization
Simple standardization is performed here along each column. Log-standardization on the nuclide concentrations and current values gave a massive improvement to the model accuracy for me. It may not be necessary for the current values.

In [5]:
type(features)

pandas.core.frame.DataFrame

In [175]:
train_split = 0.8
np.random.seed(42)

def standardize(raw_data, mean=None, std=None, axis=0):
    if mean is None:
        mean = np.mean(raw_data, axis = axis)
    if std is None:
        std = np.std(raw_data, axis = axis)
        std[ std==0 ] = 0.1
    result = (raw_data - mean) / std
    if "DataFrame" in str(type(result)):
        result = result.reset_index(drop=True)
    return result, mean, std

def unstandardize(standardized_data, mean, std):
    raw_data = (standardized_data*std)+mean
    return raw_data



num_data = len(features)
training_size = int(num_data*train_split)
testing_size = num_data - training_size
data_indices = np.arange(num_data)
training_indices = np.random.choice(num_data, training_size, replace=False)
testing_indices = data_indices[np.in1d(data_indices, training_indices, invert=True)]

training_data, data_mean, data_std = standardize(features.iloc[training_indices])
testing_data, _, _  = standardize(features.iloc[testing_indices], mean=data_mean, std=data_std)


training_flux, flux_mean, flux_std = standardize(flux_targets.iloc[training_indices])
testing_flux, _, _  = standardize(flux_targets.iloc[testing_indices], mean=flux_mean, std=flux_std)


training_final_nuclides, final_nuclides_mean, final_nuclides_std = standardize(burnup_targets.iloc[training_indices])
testing_final_nuclides, _, _  = standardize(burnup_targets.iloc[testing_indices], mean=final_nuclides_mean, std=final_nuclides_std)


print(np.shape(training_data))
print(np.shape(testing_data))
print(np.shape(training_flux))
print(np.shape(testing_flux))
print(np.shape(training_final_nuclides))
print(np.shape(testing_final_nuclides))

(160, 1330)
(40, 1330)
(160, 70)
(40, 70)
(160, 1319)
(40, 1319)


# Flux Model Creation

In [22]:
current_features = ['current_E1', 'current_E2','current_E3','current_E4','current_E5','current_E6',
            'current_E7', 'current_E8', 'current_E9', 'current_E10',
            'current_E11', 'current_E12', 'current_E13', 'current_E14',
            'current_E15', 'current_E16','current_E17','current_E18']
forced_features = ['U235', 'U238', 'temperature']
best_params = {'max_depth': 20, 
               'n_estimators': 1000, 
               'n_jobs': 20,} 
flux_model = RandomForestRegressor(random_state=0)
flux_model.set_params(**best_params)
flux_model.fit(training_data, training_flux)
flux_model_test_score = flux_model.score(testing_data, testing_flux)
print(f"RFR score: {flux_model_test_score}")
feature_labels = training_data.columns
importances = flux_model.feature_importances_

nuclides_to_use = sum( importances > np.mean(importances)/4 )
indices = np.argsort(importances)[-nuclides_to_use:]
top_importance_vals = importances[indices]
top_features = feature_labels[indices]
flux_top_features = list(set( list(top_features) + current_features + forced_features))

print(flux_top_features)


flux_model.fit(training_data[flux_top_features], training_flux)
flux_model_test_score = flux_model.score(testing_data[flux_top_features], testing_flux)


print(f"Flux R^2 score importance: {flux_model_test_score}")

RFR score: 0.9785095163298548
['Co68', 'Ba150', 'current_E17', 'current_E12', 'Sb139', 'Cs149', 'In130_m1', 'Ni69', 'Pm153', 'U235', 'Tc113', 'Sr103', 'Co70', 'Nb104', 'current_E9', 'current_E10', 'Ra227', 'Nb104_m1', 'Mo109', 'La151', 'Cu72', 'Np239', 'Ni71', 'current_E18', 'Ba149', 'Sm149', 'Pr153', 'In120', 'Pu239', 'current_E1', 'current_E8', 'current_E14', 'current_E11', 'U238', 'current_E4', 'Np242', 'Rh120', 'Tc112', 'La150', 'power', 'current_E15', 'current_E16', 'Xe135', 'U241', 'Th235', 'In121_m1', 'current_E6', 'current_E13', 'Pu240', 'Rb99', 'current_E7', 'In135', 'Rh112', 'current_E5', 'I144', 'U239', 'current_E3', 'Mo103', 'temperature', 'Cu68', 'current_E2', 'Sr105']
Flux R^2 score importance: 0.9829479533557568


In [8]:
flux_labels = list(training_flux.columns)
predicted_testing_flux = pd.DataFrame(flux_model.predict(testing_data[top_features]), 
                                      columns=testing_flux.columns)
predicted_training_flux = pd.DataFrame(flux_model.predict(training_data[top_features]), 
                                       columns=training_flux.columns)

combined_training_features = training_data.join(predicted_training_flux).drop(columns=current_features)
combined_testing_features = testing_data.join(predicted_testing_flux).drop(columns=current_features)
display(combined_training_features.iloc[0:5])

,H3,C12,C13,O16,Se74,Se80,Se82,Br81,Kr80,Kr82,...,8.92150E-02,1.47000E-01,2.42750E-01,4.01250E-01,6.60500E-01,1.08700E+00,1.79200E+00,2.95500E+00,4.87225E+00,5.00000E+36
0,-0.018371,-2.521741,2.655506,-2.679620,2.823249,2.250407,2.261060,2.260806,2.624646,2.993391,...,0.828583,0.823906,0.864011,0.978728,1.091303,1.136556,1.166252,1.224748,1.254263,1.275837
1,-0.124717,-0.055601,-0.160112,0.206401,-0.254265,0.201662,0.197560,0.191755,-0.182400,-0.311687,...,-0.654716,-0.637453,-0.632995,-0.660294,-0.682926,-0.692538,-0.716087,-0.755596,-0.758188,-0.736417
2,-0.189010,0.188048,-0.151820,0.143444,-0.362388,-0.097589,-0.100631,-0.101460,-0.287427,-0.383702,...,0.976526,0.944526,0.940442,0.980315,1.013530,1.022230,1.034116,1.050369,1.062682,1.058056
3,0.524044,-2.229200,2.249558,-2.254990,2.357219,2.062857,2.068680,2.068890,2.282226,2.480054,...,1.227278,1.236271,1.266246,1.297084,1.363196,1.446939,1.494989,1.499909,1.514051,1.541931
4,-0.189118,0.766020,-0.746467,0.737234,-0.625072,-0.865848,-0.863708,-0.862115,-0.684561,-0.573586,...,-1.116398,-1.117582,-1.121089,-1.126158,-1.134153,-1.157228,-1.177494,-1.190633,-1.178936,-1.140773


In [9]:
combined_training_features['1.47000E-01']

0      0.823906
1     -0.637453
2      0.944526
3      1.236271
4     -1.117582
         ...   
155    0.948088
156   -0.508796
157   -0.838983
158   -1.068715
159    0.043641
Name: 1.47000E-01, Length: 160, dtype: float64

# Burnup Model Creation - All Nuclides at Once

In [2]:
chain_file = "/global/home/users/ikolaja/openmc_data/chain_endfb71_pwr.xml"
chain = openmc.deplete.Chain.from_xml(chain_file)
serpent_chain = chain.reduce(list(testing_final_nuclides.columns),0)


NameError: name 'testing_final_nuclides' is not defined

In [ ]:
serpent_chain

In [14]:
def get_feature_if_exists(Z,A,nuclide_labels):
    z_map = {'Ac': 89, 'Ag': 47, 'Al': 13, 'Am': 95, 'Ar': 18, 'As': 33, 'At': 85, 'Au': 79, 'B': 5, 'Ba': 56, 'Be': 4,
             'Bh': 107, 'Bi': 83, 'Bk': 97, 'Br': 35, 'C': 6, 'Ca': 20, 'Cd': 48, 'Ce': 58,
             'Cf': 98, 'Cl': 17, 'Cm': 96, 'Co': 27, 'Cr': 24, 'Cs': 55, 'Cu': 29, 'Ds': 110, 'Db': 105, 'Dy': 66, 'Er': 68,
             'Es': 99, 'Eu': 63, 'F': 9, 'Fe': 26, 'Fm': 100, 'Fr': 87, 'Ga': 31, 'Gd':
                 64, 'Ge': 32, 'H': 1, 'He': 2, 'Hf': 72, 'Hg': 80, 'Ho': 67, 'Hs': 108, 'I': 53, 'In': 49, 'Ir': 77,
             'K': 19, 'Kr': 36, 'La': 57, 'Li': 3, 'Lr': 103, 'Lu': 71, 'Md': 101, 'Mg': 12, 'Mn':
                 25, 'Mo': 42, 'Mt': 109, 'N': 7, 'Na': 11, 'Nb': 41, 'Nd': 60, 'Ne': 10, 'Ni': 28, 'No': 102, 'Np': 93,
             'O': 8, 'Os': 76, 'P': 15, 'Pa': 91, 'Pb': 82, 'Pd': 46, 'Pm': 61, 'Po': 84, 'Pr':
                 59, 'Pt': 78, 'Pu': 94, 'Ra': 88, 'Rb': 37, 'Re': 75, 'Rf': 104, 'Rg': 111, 'Rh': 45, 'Rn': 86, 'Ru': 44,
             'S': 16, 'Sb': 51, 'Sc': 21, 'Se': 34, 'Sg': 106, 'Si': 14, 'Sm': 62, 'Sn': 50,
             'Sr': 38, 'Ta': 73, 'Tb': 65, 'Tc': 43, 'Te': 52, 'Th': 90, 'Ti': 22, 'Tl': 81, 'Tm': 69, 'U': 92, 'V': 23,
             'W': 74, 'Xe': 54, 'Y': 39, 'Yb': 70, 'Zn': 30, 'Zr': 40}
    z_map_inv = {v: k for k, v in z_map.items()}
    symbol = z_map_inv[Z]
    isotope = f"{symbol}{A}"
    matches = []
    if isotope in nuclide_labels:
        matches += [isotope]
    if isotope+"_m1" in nuclide_labels:
        matches += [isotope+"_m1"]
    if isotope+"_m2" in nuclide_labels:
        matches += [isotope+"_m2"]
    if isotope+"_m3" in nuclide_labels:
        matches += [isotope+"_m3"]
    return matches

In [15]:
serpent_chain.reactions

# 16, 17, 37, 107, 102, 103, 18

['(n,2n)', '(n,3n)', '(n,4n)', '(n,a)', '(n,gamma)', '(n,p)', 'fission']

In [275]:
list(set(old) - set(feeder_dict["Cu68"]))

[]

In [277]:
feeder_dict["Cu68"]

['Cu68',
 'Th232',
 'Pu236',
 'Am243',
 'Pa231',
 'U235',
 'U233',
 'Np236',
 'Cm247',
 'Am241',
 'Np239',
 'Pu242',
 'Np238',
 'Th227',
 'Pu239',
 'Cm250',
 'Th230',
 'Pu241',
 'Cm244',
 'U238',
 'Pa233',
 'U237',
 'Pu243',
 'Th233',
 'Cm245',
 'U241',
 'Th229',
 'Cm249',
 'Am242',
 'Np235',
 'Np237',
 'Th234',
 'Pa232',
 'U239',
 'Am244',
 'Th228',
 'Cm240',
 'Cu69',
 'Cu70',
 'Cu70_m1',
 'Cu70_m2',
 'Cu71',
 'Ga69',
 'Zn67',
 'Cu67',
 'Ga70',
 'Ni69']

In [274]:
old

['Cu68',
 'Cu68_m1',
 'Th232',
 'Pu236',
 'Am243',
 'Pa231',
 'U235',
 'U233',
 'Np236',
 'Cm247',
 'Am241',
 'Np239',
 'Pu242',
 'Np238',
 'Th227',
 'Pu239',
 'Cm250',
 'Th230',
 'Pu241',
 'Cm244',
 'U238',
 'Pa233',
 'U237',
 'Pu243',
 'Th233',
 'Cm245',
 'U241',
 'Th229',
 'Cm249',
 'Am242',
 'Np235',
 'Np237',
 'Th234',
 'Pa232',
 'U239',
 'Am244',
 'Th228',
 'Cm240',
 'Cu69',
 'Cu70',
 'Cu70_m1',
 'Cu70_m2',
 'Cu71',
 'Ga69',
 'Zn67',
 'Cu67',
 'Ga70',
 'Ni69']

In [278]:
fission_isotopes = []
available_inputs = list(training_data.drop(columns=non_concentration_columns))
nuclide_labels = list(burnup_targets)
for reaction in list(xs_targets.columns):
    MT = reaction.split("-")[1]
    if MT == '18':
        isotope = reaction.split("-")[0]
        try:
            isotope = rename_map[isotope]
        except:
            isotope = isotope[:-1]
            isotope += "<lib>"
            isotope = rename_map[isotope]
        z,a = particle_counts[isotope]
        fission_isotopes += get_feature_if_exists(z,a,isotope)
fission_isotopes = list(set(fission_isotopes))

feeder_dict = {}
for nuclide in nuclide_labels:
    starting_z,starting_a = particle_counts[nuclide]
    if nuclide in available_inputs:
        feeder_dict[nuclide] = get_feature_if_exists(starting_z, starting_a, nuclide)
    else:
        feeder_dict[nuclide] = []
    print(nuclide)
    if starting_z < 89 and starting_z > 20:
        feeder_dict[nuclide] += fission_isotopes
    # n,2n
    feeder_dict[nuclide] += get_feature_if_exists(starting_z, starting_a+1,available_inputs)
    # n,3n
    feeder_dict[nuclide] += get_feature_if_exists(starting_z, starting_a+2,available_inputs)
    # n,4n
    feeder_dict[nuclide] += get_feature_if_exists(starting_z, starting_a+3,available_inputs)
    # n,a
    feeder_dict[nuclide] += get_feature_if_exists(starting_z+2, starting_a+1,available_inputs)
    # n,p
    feeder_dict[nuclide] += get_feature_if_exists(starting_z+1, starting_a-1,available_inputs)
    # n,g
    feeder_dict[nuclide] += get_feature_if_exists(starting_z, starting_a-1,available_inputs)
    # alpha decay
    feeder_dict[nuclide] += get_feature_if_exists(starting_z+2, starting_a+2,available_inputs)
    # beta decay
    if starting_z>1:
        feeder_dict[nuclide] += get_feature_if_exists(starting_z-1, starting_a+1,available_inputs)
    print(feeder_dict[nuclide])

H1
['H1', 'H2', 'H3']
H2
['H2', 'H3', 'H1']
H3
['H3', 'H2']
He3
['He3', 'He4']
He4
['He4', 'He3']
Be9
['Be9']
B11
['B11', 'B12']
B12
['B12', 'B11']
C12
['C12', 'C13']
C13
['C13', 'C12']
O16
['O16']
Cr66
['Cr66', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Cr67', 'Fe67', 'Fe68']
Cr67
['Cr67', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Fe68', 'Mn66', 'Cr66', 'Fe69']
Mn66
['Mn66', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247'

['Ru107', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Ru108', 'Ru109', 'Ru110', 'Pd108', 'Rh106', 'Rh106_m1', 'Ru106', 'Pd109', 'Pd109_m1', 'Tc108']
Ru108
['Ru108', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Ru109', 'Ru110', 'Ru111', 'Pd109', 'Pd109_m1', 'Rh107', 'Ru107', 'Pd110', 'Tc109']
Ru109
['Ru109', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 

['Ba145', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Ba146', 'Ba147', 'Ba148', 'Ce146', 'La144', 'Ba144', 'Ce147', 'Cs146']
Ba146
['Ba146', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'Ba147', 'Ba148', 'Ba149', 'Ce147', 'La145', 'Ba145', 'Ce148', 'Cs147']
Ba147
['Ba147', 'Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', '

['Np235', 'Np236', 'Np236_m1', 'Np237', 'Np238', 'Pu234', 'Np234', 'U236']
Np236
['Np236', 'Np237', 'Np238', 'Np239', 'Pu235', 'Np235', 'Am238', 'U237']
Np236_m1
['Np236', 'Np236_m1', 'Np237', 'Np238', 'Np239', 'Pu235', 'Np235', 'Am238', 'U237']
Np237
['Np237', 'Np238', 'Np239', 'Np240', 'Np240_m1', 'Am238', 'Pu236', 'Np236', 'Np236_m1', 'Am239', 'U238']
Np238
['Np238', 'Np239', 'Np240', 'Np240_m1', 'Np241', 'Am239', 'Pu237', 'Pu237_m1', 'Np237', 'Am240', 'U239']
Np239
['Np239', 'Np240', 'Np240_m1', 'Np241', 'Np242', 'Am240', 'Pu238', 'Np238', 'Am241', 'U240']
Np240
['Np240', 'Np241', 'Np242', 'Am241', 'Pu239', 'Np239', 'Am242', 'Am242_m1', 'U241']
Np240_m1
['Np240', 'Np240_m1', 'Np241', 'Np242', 'Am241', 'Pu239', 'Np239', 'Am242', 'Am242_m1', 'U241']
Np241
['Np241', 'Np242', 'Am242', 'Am242_m1', 'Pu240', 'Np240', 'Np240_m1', 'Am243', 'U242']
Np242
['Np242', 'Am243', 'Pu241', 'Np241', 'Am244', 'Am244_m1']
Pu234
['Pu234', 'Pu235', 'Pu236', 'Pu237', 'Pu237_m1', 'Np235']
Pu235
['Pu235', '

In [251]:
burnup_models = {}
burnup_r2_scores = {}
burnup_mape_scores = {}

In [291]:
list(burnup_models.keys())[:10]

['Ac230',
 'Nd139',
 'Th224',
 'Pm142',
 'Ra220',
 'Cd104',
 'In110_m1',
 'Sn110',
 'Sm142',
 'Rn216']

In [292]:
forced_features = flux_labels+["power","temperature","depletion_time"]
size = (32,16)

for target_label in list(burnup_models.keys())[:10]:#["Cu68"]:#nuclide_labels:
    starting_z,starting_a = particle_counts[target_label]
    #if os.path.isfile(f"ml_models/burnup_nuclides/{target_label}.pkl"):
    #    continue
    top_features = feeder_dict[target_label]+forced_features 
    print(target_label)
    print(top_features)
    
    train_len = combined_training_features[top_features].shape[0]
    test_len = combined_testing_features.shape[0]
    num_outputs = 1
    num_inputs = combined_training_features[top_features].shape[1]
    
    X_train = np.array(combined_training_features[top_features]).reshape(train_len, -1, 1)
    X_test = np.array(combined_testing_features[top_features]).reshape(test_len, -1, 1)
    y_train = np.array(training_final_nuclides[target_label]).reshape(train_len, -1, 1)
    y_test = np.array(testing_final_nuclides[target_label]).reshape(test_len, -1, 1)
    

    

    if type(size) == int:
        nn_model = Sequential()
        nn_model.add(Dense(size, input_dim=num_inputs, activation='relu'))
        nn_model.add(Dense(num_outputs, activation='linear'))
    else:
        nn_model = Sequential()
        nn_model.add(Dense(size[0], input_dim=num_inputs, activation='relu'))
        nn_model.add(Dense(size[1], activation='relu')) 
        nn_model.add(Dense(num_outputs, activation='linear'))  

    batch_size = 512
    learning_rate = 0.001
    epochs = 400

    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    nn_model.compile(loss='mse', optimizer=optimizer)
    history = nn_model.fit(X_train, y_train, 
                            epochs=epochs, batch_size=batch_size, verbose=0, 
                            validation_data=(X_test, y_test))


    y_predict = nn_model.predict(X_test)
    column_predict = y_predict[:,0]
    column_truth = testing_final_nuclides[target_label]
    r2_score_val = r2_score(column_truth, column_predict)
    mape_score_val = mean_absolute_percentage_error(column_truth, column_predict)*100
    print(f"score: R^2={r2_score_val}   MAPE={mape_score_val}")
    print(f"Training loss: {history.history['loss'][-1]}")
    print(f"Validation loss: {history.history['val_loss'][-1]}")

    burnup_models[target_label] = nn_model
    burnup_r2_scores[target_label] = r2_score_val
    burnup_mape_scores[target_label] = mape_score_val

Ac230
['Ac231', 'Ac232', 'Pa231', 'Th229', 'Ac229', 'Pa232', '2.50000E-09', '7.50000E-09', '1.25000E-08', '1.75000E-08', '2.25000E-08', '2.75000E-08', '3.25000E-08', '3.85000E-08', '4.60000E-08', '5.40000E-08', '6.25000E-08', '7.35000E-08', '9.00000E-08', '1.20000E-07', '1.60000E-07', '2.00000E-07', '2.35000E-07', '2.65000E-07', '2.90000E-07', '3.10000E-07', '3.35000E-07', '3.75000E-07', '4.50000E-07', '5.62500E-07', '7.02500E-07', '8.15000E-07', '8.80000E-07', '9.30000E-07', '9.61000E-07', '9.84000E-07', '1.00800E-06', '1.03250E-06', '1.05800E-06', '1.08400E-06', '1.11000E-06', '1.13650E-06', '1.22500E-06', '1.40000E-06', '1.67750E-06', '1.97750E-06', '2.35000E-06', '2.95000E-06', '3.65000E-06', '6.93850E-06', '1.29225E-05', '2.18340E-05', '3.78760E-05', '6.17767E-05', '1.12115E-04', '2.57995E-04', '6.37080E-04', '1.16600E-03', '1.83227E-03', '2.87927E-03', '4.50955E-03', '7.30900E-03', '1.20740E-02', '1.99050E-02', '3.28150E-02', '5.41400E-02', '8.92150E-02', '1.47000E-01', '2.42750E

2/2 [==============================] - 0s 3ms/step
score: R^2=-448.58892071271083   MAPE=359.2144998774966
Training loss: 0.0005884187412448227
Validation loss: 1.3351799249649048
In110_m1
['Th232', 'Pu236', 'Am243', 'Pa231', 'U235', 'U233', 'Np236', 'Cm247', 'Am241', 'Np239', 'Pu242', 'Np238', 'Th227', 'Pu239', 'Cm250', 'Th230', 'Pu241', 'Cm244', 'U238', 'Pa233', 'U237', 'Pu243', 'Th233', 'Cm245', 'U241', 'Th229', 'Cm249', 'Am242', 'Np235', 'Np237', 'Th234', 'Pa232', 'U239', 'Am244', 'Th228', 'Cm240', 'In111', 'In111_m1', 'In112', 'In112_m1', 'In113', 'In113_m1', 'In109', 'Cd111', 'Cd111_m1', '2.50000E-09', '7.50000E-09', '1.25000E-08', '1.75000E-08', '2.25000E-08', '2.75000E-08', '3.25000E-08', '3.85000E-08', '4.60000E-08', '5.40000E-08', '6.25000E-08', '7.35000E-08', '9.00000E-08', '1.20000E-07', '1.60000E-07', '2.00000E-07', '2.35000E-07', '2.65000E-07', '2.90000E-07', '3.10000E-07', '3.35000E-07', '3.75000E-07', '4.50000E-07', '5.62500E-07', '7.02500E-07', '8.15000E-07', '8.80000E

In [280]:
burnup_targets["Ac230"]

0      1.503534e-39
1      6.927370e-41
2      6.290185e-41
3      3.438837e-41
4      3.441794e-39
           ...     
195    9.682607e-41
196    6.596148e-40
197    0.000000e+00
198    1.934980e-40
199    0.000000e+00
Name: Ac230, Length: 200, dtype: float64

In [294]:
for key in burnup_models.keys():
    with open(f"ml_models/burnup_nuclides/{key}.pkl", 'wb') as f:
        model_data = pickle.dumps(burnup_models[key])
        f.write(model_data)
    #with open(f"ml_models/burnup_nuclides/{key}_score.csv", "w") as f:
     #   f.write(f"r2,mape\n{burnup_r2_scores[key]},{burnup_mape_scores[key]}")

In [295]:
target_label = "U235"
top_features = feeder_dict[target_label]+forced_features 
print(target_label)
print(top_features)
train_len = combined_training_features[top_features].shape[0]
test_len = combined_testing_features.shape[0]
num_outputs = 1
num_inputs = combined_training_features[top_features].shape[1]
X_train = np.array(combined_training_features[top_features]).reshape(train_len, -1, 1)
X_test = np.array(combined_testing_features[top_features]).reshape(test_len, -1, 1)

Y_pred = burnup_models["U235"].predict(X_test)

U235
['U235', 'U236', 'U237', 'U238', 'Pu236', 'Np234', 'U234', 'Pu237', 'Pu237_m1', '2.50000E-09', '7.50000E-09', '1.25000E-08', '1.75000E-08', '2.25000E-08', '2.75000E-08', '3.25000E-08', '3.85000E-08', '4.60000E-08', '5.40000E-08', '6.25000E-08', '7.35000E-08', '9.00000E-08', '1.20000E-07', '1.60000E-07', '2.00000E-07', '2.35000E-07', '2.65000E-07', '2.90000E-07', '3.10000E-07', '3.35000E-07', '3.75000E-07', '4.50000E-07', '5.62500E-07', '7.02500E-07', '8.15000E-07', '8.80000E-07', '9.30000E-07', '9.61000E-07', '9.84000E-07', '1.00800E-06', '1.03250E-06', '1.05800E-06', '1.08400E-06', '1.11000E-06', '1.13650E-06', '1.22500E-06', '1.40000E-06', '1.67750E-06', '1.97750E-06', '2.35000E-06', '2.95000E-06', '3.65000E-06', '6.93850E-06', '1.29225E-05', '2.18340E-05', '3.78760E-05', '6.17767E-05', '1.12115E-04', '2.57995E-04', '6.37080E-04', '1.16600E-03', '1.83227E-03', '2.87927E-03', '4.50955E-03', '7.30900E-03', '1.20740E-02', '1.99050E-02', '3.28150E-02', '5.41400E-02', '8.92150E-02', 

In [296]:
r2_score(Y_pred[:,0], testing_final_nuclides["U235"])

0.9877901617760878

In [297]:
Y_pred[:,0]

array([ 0.88425094,  0.68976086, -0.0606676 ,  0.15704387,  0.520678  ,
        0.4997814 ,  0.24737182, -0.39396384,  0.45340136, -2.1749022 ,
       -1.7507936 , -0.21490821,  0.71913004,  0.64048445,  0.9404525 ,
        0.57225245,  0.15266472,  0.96239465,  0.98849386,  0.9064822 ,
       -0.6526013 ,  0.46626997, -2.4612396 ,  0.60780245,  0.0074949 ,
        0.8964702 ,  0.85919386,  0.7809638 ,  0.9587676 ,  1.071464  ,
        0.97011477,  0.50638443, -1.7923672 , -0.11430722,  0.0092184 ,
       -1.2279632 , -1.7221508 , -0.48233423, -1.1836818 ,  1.0261141 ],
      dtype=float32)

In [298]:
top_feature_dict["Cu68"]

['Cu68',
 'Th232',
 'Pu236',
 'Am243',
 'Pa231',
 'U235',
 'U233',
 'Np236',
 'Cm247',
 'Am241',
 'Np239',
 'Pu242',
 'Np238',
 'Th227',
 'Pu239',
 'Cm250',
 'Th230',
 'Pu241',
 'Cm244',
 'U238',
 'Pa233',
 'U237',
 'Pu243',
 'Th233',
 'Cm245',
 'U241',
 'Th229',
 'Cm249',
 'Am242',
 'Np235',
 'Np237',
 'Th234',
 'Pa232',
 'U239',
 'Am244',
 'Th228',
 'Cm240',
 'Cu69',
 'Cu70',
 'Cu70_m1',
 'Cu70_m2',
 'Cu71',
 'Ga69',
 'Zn67',
 'Cu67',
 'Ga70',
 'Ni69',
 '2.50000E-09',
 '7.50000E-09',
 '1.25000E-08',
 '1.75000E-08',
 '2.25000E-08',
 '2.75000E-08',
 '3.25000E-08',
 '3.85000E-08',
 '4.60000E-08',
 '5.40000E-08',
 '6.25000E-08',
 '7.35000E-08',
 '9.00000E-08',
 '1.20000E-07',
 '1.60000E-07',
 '2.00000E-07',
 '2.35000E-07',
 '2.65000E-07',
 '2.90000E-07',
 '3.10000E-07',
 '3.35000E-07',
 '3.75000E-07',
 '4.50000E-07',
 '5.62500E-07',
 '7.02500E-07',
 '8.15000E-07',
 '8.80000E-07',
 '9.30000E-07',
 '9.61000E-07',
 '9.84000E-07',
 '1.00800E-06',
 '1.03250E-06',
 '1.05800E-06',
 '1.08400E-06

In [299]:
top_feature_dict = {}
forced_features = flux_labels+["power","temperature","depletion_time"]
for target_label in nuclide_labels:
    top_feature_dict[target_label] = feeder_dict[target_label]+forced_features
top_feature_dict["H2"]
#with open("ml_models/burnup_feature_dict.json", "w") as outfile: 
#    json.dump(top_feature_dict, outfile)

['H2',
 'H3',
 'H1',
 '2.50000E-09',
 '7.50000E-09',
 '1.25000E-08',
 '1.75000E-08',
 '2.25000E-08',
 '2.75000E-08',
 '3.25000E-08',
 '3.85000E-08',
 '4.60000E-08',
 '5.40000E-08',
 '6.25000E-08',
 '7.35000E-08',
 '9.00000E-08',
 '1.20000E-07',
 '1.60000E-07',
 '2.00000E-07',
 '2.35000E-07',
 '2.65000E-07',
 '2.90000E-07',
 '3.10000E-07',
 '3.35000E-07',
 '3.75000E-07',
 '4.50000E-07',
 '5.62500E-07',
 '7.02500E-07',
 '8.15000E-07',
 '8.80000E-07',
 '9.30000E-07',
 '9.61000E-07',
 '9.84000E-07',
 '1.00800E-06',
 '1.03250E-06',
 '1.05800E-06',
 '1.08400E-06',
 '1.11000E-06',
 '1.13650E-06',
 '1.22500E-06',
 '1.40000E-06',
 '1.67750E-06',
 '1.97750E-06',
 '2.35000E-06',
 '2.95000E-06',
 '3.65000E-06',
 '6.93850E-06',
 '1.29225E-05',
 '2.18340E-05',
 '3.78760E-05',
 '6.17767E-05',
 '1.12115E-04',
 '2.57995E-04',
 '6.37080E-04',
 '1.16600E-03',
 '1.83227E-03',
 '2.87927E-03',
 '4.50955E-03',
 '7.30900E-03',
 '1.20740E-02',
 '1.99050E-02',
 '3.28150E-02',
 '5.41400E-02',
 '8.92150E-02',
 '1

Next we save the models and standardization parameters so the model can be used in the simulation.

In [23]:
data_mean.to_csv("ml_models/burnup_features_mean.csv", header=True)
data_std.to_csv("ml_models/burnup_features_std.csv", header=True)
flux_mean.to_csv("ml_models/burnup_flux_mean.csv", header=True)
flux_std.to_csv("ml_models/burnup_flux_std.csv", header=True)
final_nuclides_mean.to_csv("ml_models/burnup_target_mean.csv", header=True)
final_nuclides_std.to_csv("ml_models/burnup_target_std.csv", header=True)

# Create DepletionWrapper Object


In [257]:
for key in nuclide_labels:
    with open(f"ml_models/burnup_nuclides/{key}.pkl", 'rb') as f:
        burnup_models[key] = pickle.loads(f.read())

In [300]:
combined_top_feature_dict = top_feature_dict.copy()
combined_top_feature_dict["flux"] = flux_top_features
combined_target_mean = pd.concat([final_nuclides_mean,flux_mean]) 
combined_target_std = pd.concat([final_nuclides_std,flux_std])

In [301]:
class DepletionWrapper():
    def __init__(self, flux_model, depletion_model_dict, input_feature_dict, 
                 data_mean, data_std, target_mean, target_std, 
                 accuracy_dict={}, flux_model_library="sklearn", depletion_model_library="keras",
                 flux_labels = flux_labels):
        self.flux_model = flux_model # dictionary mapping target to model object
        self.depletion_models = depletion_model_dict # dictionary mapping target to model object
        self.input_feature_dict = input_feature_dict # dictionary mapping target to lists of best features
        self.accuracy_accuracy = accuracy_dict # dictionary mapping isotope to R^2 values on final train
        self.flux_model_library = flux_model_library
        self.depletion_model_library = depletion_model_library
        self.data_mean = data_mean
        self.data_std = data_std
        self.target_mean = target_mean
        self.target_std = target_std
        self.flux_labels = flux_labels

    def standardize_data(self, raw_data, axis=0):
        aligned_data = raw_data[list(self.data_mean.axes[0])]
        result = (aligned_data - self.data_mean) / self.data_std
        return result

    def unstandardize_target(self, standardized_target, label):
        raw_target = (standardized_target*self.target_std[label])+self.target_mean[label]
        return raw_target

    def predict(self, features_raw, columns="all", debug=0):
        results = {}
        results_std = {}
        features_std = self.standardize_data(features_raw).reset_index(drop=True)
        if columns == "all":
            predict_columns = self.depletion_models.keys()
        else:
            assert type(columns) == list
            predict_columns = columns
        # Flux Model
        if "tensorflow" == self.flux_model_library:
            for target in self.flux_model.keys():
                model = self.flux_model[target]
                input_features = self.input_feature_dict[target]
                
                X_std = features_std[input_features]
                data_len = X_std.shape[0]
                num_inputs = X_std.shape[1]
                X_std = np.array(X_std).reshape(data_len, -1, 1)
                if debug > 0:
                    print(f"Predicting {target}")
                Y_std = model.predict(X_std)[:,0]
                Y = self.unstandardize_target(Y_std, target)
                results[target] = Y
                results_std[target] = Y_std
        
        elif "sklearn" == self.flux_model_library:
            input_features = self.input_feature_dict["flux"]
            X_std = features_std[input_features]
            data_len = X_std.shape[0]

            if debug > 0:
                print(f"Predicting flux")
            Y_std = self.flux_model.predict(X_std)
            i = 0
            for target in self.flux_labels:
                Y = self.unstandardize_target(Y_std[:,i], target)
                results[target] = Y
                results_std[target] = Y_std[:,i]
                i += 1

        # Depletion Modeling
        # Combine nuclides with newly calculated flux
        calculated_features = pd.DataFrame(results_std).reset_index(drop=True)
        features_std = pd.concat([features_std.reset_index(drop=True), calculated_features], axis=1).dropna(axis=1)
        
        

        for target in predict_columns:
            model = self.depletion_models[target]
            input_features = self.input_feature_dict[target]
            X_std = features_std[input_features]
            if "keras" == self.depletion_model_library:
                data_len = X_std.shape[0]
                num_inputs = X_std.shape[1]
                X_std = np.array(X_std).reshape(data_len, -1, 1)

                if debug > 0:
                    print(f"Predicting {target}")
                Y_std = model.predict(X_std)[:,0]
                Y = self.unstandardize_target(Y_std, target)
                results[target] = Y
                results_std[target] = Y_std
            elif "sklearn" == self.depletion_model_library:
                data_len = X_std.shape[0]

                if debug > 0:
                    print(f"Predicting {target}")
                Y_std = model.predict(X_std)
                Y = self.unstandardize_target(Y_std, target)
                results[target] = Y
                results_std[target] = Y_std
        return pd.DataFrame(results)

In [302]:
depletion_model = DepletionWrapper(flux_model, burnup_models, combined_top_feature_dict, 
                 data_mean, data_std, combined_target_mean, combined_target_std)

predicted_Y, predicted_Y_std, features_std = depletion_model.predict(features.iloc[testing_indices], debug=1)

,H3,C12,C13,O16,Se74,Se80,Se82,Br81,Kr80,Kr82,...,current_E12,current_E13,current_E14,current_E15,current_E16,current_E17,current_E18,power,depletion_time,temperature
0,-0.189054,0.766331,-0.728293,0.716302,-0.625551,-0.861218,-0.859110,-0.857573,-0.682637,-0.573465,...,-1.002975,-1.002225,-1.027495,-1.024259,-1.027711,-1.025099,-1.012605,-0.914625,0.167687,0.0
1,-0.189188,0.745004,-0.555964,0.520102,-0.625117,-0.808498,-0.806718,-0.806108,-0.665072,-0.570843,...,2.727159,2.605617,2.793832,2.567948,2.855335,2.552748,2.129834,4.639019,0.167687,0.0
2,-0.187886,0.194067,-0.148824,0.134862,-0.388531,-0.115788,-0.118721,-0.119609,-0.294421,-0.393727,...,1.855472,1.890287,1.660518,1.538886,1.714389,1.963009,1.811383,1.561776,0.167687,0.0
3,-0.187169,0.380328,-0.232184,0.201052,-0.454424,-0.319973,-0.321654,-0.321507,-0.400984,-0.464870,...,1.256892,1.134946,1.411671,1.312768,1.024115,1.364689,1.354640,0.782450,0.167687,0.0
4,-0.189167,0.537130,-0.555547,0.563719,-0.565474,-0.501768,-0.501704,-0.503441,-0.548771,-0.546062,...,-0.822007,-0.854972,-0.829525,-0.848111,-0.846688,-0.824229,-0.855583,-0.698650,0.167687,0.0
5,-0.185291,0.513810,-0.618236,0.639834,-0.579235,-0.468286,-0.468210,-0.471507,-0.538269,-0.541244,...,-0.918743,-0.915139,-0.940969,-0.911097,-0.909885,-0.938012,-0.933641,-0.560903,0.167687,0.0
6,-0.189165,0.443279,-0.389450,0.374531,-0.518778,-0.431450,-0.432322,-0.432739,-0.478530,-0.492628,...,-0.044499,-0.027923,-0.007360,-0.181873,-0.075618,-0.011054,-0.208411,-0.588397,-1.291359,0.0
7,-0.177769,-0.208201,0.333992,-0.353923,-0.001508,0.387836,0.382206,0.381387,0.080788,-0.138402,...,1.049964,0.889349,1.080614,1.130437,1.235397,1.082734,0.947536,0.805245,-0.642839,0.0
8,-0.187314,0.532082,-0.531186,0.528172,-0.561989,-0.496254,-0.496234,-0.498145,-0.532661,-0.539616,...,-0.683561,-0.687594,-0.729375,-0.757147,-0.709375,-0.720092,-0.748603,-0.832674,-1.291359,0.0
9,1.422746,-2.215139,2.166168,-2.159477,2.302787,2.038833,2.043396,2.043194,2.286886,2.299160,...,1.449550,1.525157,1.504396,1.589849,1.490152,1.583264,1.315908,0.793268,0.167687,0.0


Predicting flux
Predicting Ac230
2/2 [==============================] - 0s 2ms/step
Predicting Nd139
2/2 [==============================] - 0s 2ms/step
Predicting Th224
2/2 [==============================] - 0s 2ms/step
Predicting Pm142
2/2 [==============================] - 0s 2ms/step
Predicting Ra220
2/2 [==============================] - 0s 2ms/step
Predicting Cd104
2/2 [==============================] - 0s 2ms/step
Predicting In110_m1
2/2 [==============================] - 0s 2ms/step
Predicting Sn110
2/2 [==============================] - 0s 2ms/step
Predicting Sm142
2/2 [==============================] - 0s 2ms/step
Predicting Rn216
2/2 [==============================] - 0s 2ms/step
Predicting H1
2/2 [==============================] - 0s 2ms/step
Predicting H2
2/2 [==============================] - 0s 2ms/step
Predicting H3
2/2 [==============================] - 0s 2ms/step
Predicting He3
2/2 [==============================] - 0s 2ms/step
Predicting He4
2/2 [====================

2/2 [==============================] - 0s 2ms/step
Predicting Kr90
2/2 [==============================] - 0s 2ms/step
Predicting Kr91
2/2 [==============================] - 0s 2ms/step
Predicting Kr92
2/2 [==============================] - 0s 2ms/step
Predicting Kr93
2/2 [==============================] - 0s 2ms/step
Predicting Kr94
2/2 [==============================] - 0s 2ms/step
Predicting Kr95
2/2 [==============================] - 0s 2ms/step
Predicting Kr96
2/2 [==============================] - 0s 2ms/step
Predicting Kr97
2/2 [==============================] - 0s 2ms/step
Predicting Kr98
2/2 [==============================] - 0s 2ms/step
Predicting Kr99
2/2 [==============================] - 0s 2ms/step
Predicting Kr100
2/2 [==============================] - 0s 2ms/step
Predicting Rb79
2/2 [==============================] - 0s 2ms/step
Predicting Rb81
2/2 [==============================] - 0s 2ms/step
Predicting Rb83
2/2 [==============================] - 0s 2ms/step
Predicting

2/2 [==============================] - 0s 3ms/step
Predicting Rh105
2/2 [==============================] - 0s 3ms/step
Predicting Rh105_m1
2/2 [==============================] - 0s 2ms/step
Predicting Rh106
2/2 [==============================] - 0s 2ms/step
Predicting Rh106_m1
2/2 [==============================] - 0s 2ms/step
Predicting Rh107
2/2 [==============================] - 0s 2ms/step
Predicting Rh108
2/2 [==============================] - 0s 2ms/step
Predicting Rh108_m1
2/2 [==============================] - 0s 3ms/step
Predicting Rh109
2/2 [==============================] - 0s 2ms/step
Predicting Rh110
2/2 [==============================] - 0s 3ms/step
Predicting Rh110_m1
2/2 [==============================] - 0s 3ms/step
Predicting Rh111
2/2 [==============================] - 0s 3ms/step
Predicting Rh112
2/2 [==============================] - 0s 2ms/step
Predicting Rh113
2/2 [==============================] - 0s 3ms/step
Predicting Rh114
2/2 [==============================]

2/2 [==============================] - 0s 2ms/step
Predicting Sb129
2/2 [==============================] - 0s 2ms/step
Predicting Sb130
2/2 [==============================] - 0s 2ms/step
Predicting Sb130_m1
2/2 [==============================] - 0s 2ms/step
Predicting Sb131
2/2 [==============================] - 0s 2ms/step
Predicting Sb132
2/2 [==============================] - 0s 2ms/step
Predicting Sb132_m1
2/2 [==============================] - 0s 2ms/step
Predicting Sb133
2/2 [==============================] - 0s 2ms/step
Predicting Sb134
2/2 [==============================] - 0s 2ms/step
Predicting Sb134_m1
2/2 [==============================] - 0s 2ms/step
Predicting Sb135
2/2 [==============================] - 0s 2ms/step
Predicting Sb136
2/2 [==============================] - 0s 2ms/step
Predicting Sb137
2/2 [==============================] - 0s 2ms/step
Predicting Sb138
2/2 [==============================] - 0s 3ms/step
Predicting Sb139
2/2 [==============================] - 

2/2 [==============================] - 0s 2ms/step
Predicting Cs138_m1
2/2 [==============================] - 0s 2ms/step
Predicting Cs139
2/2 [==============================] - 0s 2ms/step
Predicting Cs140
2/2 [==============================] - 0s 2ms/step
Predicting Cs141
2/2 [==============================] - 0s 2ms/step
Predicting Cs142
2/2 [==============================] - 0s 2ms/step
Predicting Cs143
2/2 [==============================] - 0s 2ms/step
Predicting Cs144
2/2 [==============================] - 0s 2ms/step
Predicting Cs145
2/2 [==============================] - 0s 2ms/step
Predicting Cs146
2/2 [==============================] - 0s 2ms/step
Predicting Cs147
2/2 [==============================] - 0s 2ms/step
Predicting Cs148
2/2 [==============================] - 0s 2ms/step
Predicting Cs149
2/2 [==============================] - 0s 2ms/step
Predicting Cs150
2/2 [==============================] - 0s 2ms/step
Predicting Cs151
2/2 [==============================] - 0s 2ms

2/2 [==============================] - 0s 2ms/step
Predicting Nd145
2/2 [==============================] - 0s 2ms/step
Predicting Nd146
2/2 [==============================] - 0s 2ms/step
Predicting Nd147
2/2 [==============================] - 0s 2ms/step
Predicting Nd148
2/2 [==============================] - 0s 2ms/step
Predicting Nd149
2/2 [==============================] - 0s 2ms/step
Predicting Nd150
2/2 [==============================] - 0s 2ms/step
Predicting Nd151
2/2 [==============================] - 0s 3ms/step
Predicting Nd152
2/2 [==============================] - 0s 2ms/step
Predicting Nd153
2/2 [==============================] - 0s 2ms/step
Predicting Nd154
2/2 [==============================] - 0s 2ms/step
Predicting Nd155
2/2 [==============================] - 0s 2ms/step
Predicting Nd156
2/2 [==============================] - 0s 2ms/step
Predicting Nd157
2/2 [==============================] - 0s 2ms/step
Predicting Nd158
2/2 [==============================] - 0s 3ms/st

2/2 [==============================] - 0s 2ms/step
Predicting Fr222
2/2 [==============================] - 0s 2ms/step
Predicting Fr223
2/2 [==============================] - 0s 2ms/step
Predicting Ra221
2/2 [==============================] - 0s 2ms/step
Predicting Ra222
2/2 [==============================] - 0s 2ms/step
Predicting Ra223
2/2 [==============================] - 0s 3ms/step
Predicting Ra224
2/2 [==============================] - 0s 2ms/step
Predicting Ra225
2/2 [==============================] - 0s 2ms/step
Predicting Ra226
2/2 [==============================] - 0s 2ms/step
Predicting Ra227
2/2 [==============================] - 0s 2ms/step
Predicting Ra228
2/2 [==============================] - 0s 2ms/step
Predicting Ra229
2/2 [==============================] - 0s 2ms/step
Predicting Ac225
2/2 [==============================] - 0s 3ms/step
Predicting Ac226
2/2 [==============================] - 0s 2ms/step
Predicting Ac227
2/2 [==============================] - 0s 2ms/st

In [303]:
#r2_score(predicted_Y['1.25000E-08'], flux_targets['1.25000E-08'].iloc[testing_indices].reset_index(drop=True))
r2_score(predicted_Y_std['U235'], testing_final_nuclides["U235"])


0.9877901617760878

In [304]:
actual_Y = burnup_targets.iloc[testing_indices].reset_index(drop=True)
final_r2_scores = {}
final_mape_scores = {}
for target_label in testing_final_nuclides.columns:
    final_r2_scores[target_label] = r2_score(actual_Y[target_label], predicted_Y[target_label])
    final_mape_scores[target_label] = mean_absolute_percentage_error(actual_Y[target_label], predicted_Y[target_label])*100
    print(f"{target_label}: r2 = {final_r2_scores[target_label]} MAPE = {final_mape_scores[target_label]}%")

H1: r2 = 0.9681988439980109 MAPE = 765.2312269357303%
H2: r2 = 0.9785758088547244 MAPE = 663.0243415628305%
H3: r2 = 0.9517184103384115 MAPE = 285.58597334868654%
He3: r2 = 0.9947112505975796 MAPE = 26.56260071549922%
He4: r2 = 0.9830182100362426 MAPE = 428.46925944272857%
Be9: r2 = 0.9916734985422886 MAPE = 141.96142757585363%
B11: r2 = 0.9978931309684864 MAPE = 3427.3800405273414%
B12: r2 = 0.6761497193311572 MAPE = 9.521368192001259e-05%
C12: r2 = 0.9932770369769944 MAPE = 1.3895290942103248e-05%
C13: r2 = 0.9930783533063924 MAPE = 0.005081105002569613%
O16: r2 = 0.9917820063264274 MAPE = 2.04510990958971e-05%
Cr66: r2 = 0.8398364983595124 MAPE = 1.8115581989402298e-08%
Cr67: r2 = 0.9061594841729714 MAPE = 1.632085151562066e-08%
Mn66: r2 = 0.927938287522349 MAPE = 1.328706409394171e-05%
Mn67: r2 = 0.9428027606240947 MAPE = 4.584976884754574e-06%
Mn68: r2 = 0.912137826500376 MAPE = 7.202722102771638e-07%
Mn69: r2 = 0.765345106827436 MAPE = 1.181204023113624e-07%
Fe65: r2 = 0.96988098

Se83: r2 = 0.9377150587076274 MAPE = 15.49577099298219%
Se83_m1: r2 = 0.9463132978877703 MAPE = 22.49561763085822%
Se84: r2 = 0.7896313881244414 MAPE = 32.911824805280894%
Se85: r2 = 0.9198317736852977 MAPE = 18.185307751042213%
Se86: r2 = 0.8137177491945949 MAPE = 18.09365391848828%
Se87: r2 = 0.867574947554705 MAPE = 20.326543613738394%
Se88: r2 = 0.9043661637786893 MAPE = 25.942121444909294%
Se89: r2 = 0.6807590724772985 MAPE = 28.951729821077166%
Se90: r2 = 0.8908339591916505 MAPE = 18.6148957964731%
Se91: r2 = 0.838995987708149 MAPE = 14.782875380565647%
Se92: r2 = 0.7185331481858095 MAPE = 0.4787386328651112%
Se93: r2 = 0.851461006021457 MAPE = 0.011731179069987092%
Se94: r2 = 0.7659168174155072 MAPE = 0.0002814594894999264%
Br75: r2 = 0.7389705376583292 MAPE = 2.6014103875453676e-08%
Br77: r2 = 0.7955972344948834 MAPE = 0.14062401019321408%
Br77_m1: r2 = 0.8498944149169234 MAPE = 0.00014287528119934762%
Br78: r2 = 0.7499673347692484 MAPE = 0.03144959829978129%
Br79: r2 = 0.78638

Mo94: r2 = 0.7452027589734569 MAPE = 127374.85144984283%
Mo95: r2 = 0.9047378463629904 MAPE = 2317742.387295706%
Mo96: r2 = 0.6901008887906999 MAPE = 876090.0756883409%
Mo97: r2 = 0.7928752019607019 MAPE = 468.9396699633961%
Mo98: r2 = 0.5956025462643575 MAPE = 2481.9145961601316%
Mo99: r2 = 0.9499224743178426 MAPE = 22.982233512955865%
Mo100: r2 = 0.9035332717098494 MAPE = 330.8966951770933%
Mo101: r2 = 0.9656323831498094 MAPE = 11.53923933114602%
Mo102: r2 = 0.9383259041210796 MAPE = 18.54947963627932%
Mo103: r2 = 0.8877778170525581 MAPE = 18.92911156146015%
Mo104: r2 = 0.8061024594244813 MAPE = 43.64707458348288%
Mo105: r2 = 0.9616703366475031 MAPE = 13.366676707429415%
Mo106: r2 = 0.736671945376955 MAPE = 25.838954635359556%
Mo107: r2 = 0.722013884203556 MAPE = 29.757917705887348%
Mo108: r2 = 0.9157684998827431 MAPE = 21.706595014030782%
Mo109: r2 = 0.8123973020846058 MAPE = 29.691723300182026%
Mo110: r2 = 0.654086234033103 MAPE = 45.314642634063716%
Mo111: r2 = 0.7613818988008338 

In120: r2 = 0.8577432168153882 MAPE = 32.383282811596075%
In120_m1: r2 = 0.9118682543687355 MAPE = 34.0364519051817%
In120_m2: r2 = 0.7960118347150543 MAPE = 45.894130656719575%
In121: r2 = 0.9514027113687211 MAPE = 21.49808307359812%
In121_m1: r2 = -0.06537891360022341 MAPE = 33.39957446612184%
In122: r2 = 0.951450743066364 MAPE = 15.074313976771112%
In122_m1: r2 = 0.7942739459265289 MAPE = 58.59755640941267%
In122_m2: r2 = 0.7178238064088472 MAPE = 26.470541063418697%
In123: r2 = 0.768298769309506 MAPE = 19.092583900798655%
In123_m1: r2 = 0.6828901263282177 MAPE = 23.270106272259365%
In124: r2 = 0.868715922464752 MAPE = 35.21524615898136%
In124_m1: r2 = 0.6359354858131763 MAPE = 38.646021070791264%
In125: r2 = 0.9130023899081767 MAPE = 27.720044378703907%
In125_m1: r2 = 0.9599662919707973 MAPE = 21.83464322023228%
In126: r2 = 0.8674585918234555 MAPE = 30.214386602390668%
In126_m1: r2 = 0.9112534533821558 MAPE = 34.659830560815486%
In127: r2 = 0.9034641514291917 MAPE = 22.062855534695

Ba133: r2 = 0.3133422619734548 MAPE = 5338.578658619879%
Ba133_m1: r2 = 0.853125261025379 MAPE = 2.9105057199751356%
Ba134: r2 = 0.7669227708568214 MAPE = 4718974746.075599%
Ba135: r2 = 0.9036462125082677 MAPE = 7501005.983857063%
Ba135_m1: r2 = 0.45297805926048507 MAPE = 17802.403912957798%
Ba136: r2 = 0.729480547464286 MAPE = 48723.83059873246%
Ba136_m1: r2 = 0.5212001476280431 MAPE = 177.89556709491777%
Ba137: r2 = 0.8931916857543187 MAPE = 32468.312484770486%
Ba137_m1: r2 = 0.9530314969764927 MAPE = 541.3418786100342%
Ba138: r2 = 0.6155605246224237 MAPE = 1855.1542322664154%
Ba139: r2 = 0.8934297356365898 MAPE = 26.156468998978045%
Ba140: r2 = 0.7268537766187314 MAPE = 32.49333224765586%
Ba141: r2 = 0.929352962445607 MAPE = 16.69650585905331%
Ba142: r2 = 0.8498091789537116 MAPE = 30.036457358857998%
Ba143: r2 = 0.7699330695223296 MAPE = 39.839487305199405%
Ba144: r2 = 0.594915137624085 MAPE = 37.915178546970374%
Ba145: r2 = 0.8812487222544293 MAPE = 17.999081261519603%
Ba146: r2 = 

Dy157: r2 = 0.900564822345312 MAPE = 0.003648488407061945%
Dy158: r2 = 0.8467387803908046 MAPE = 43.24122211817126%
Dy159: r2 = 0.611949741398973 MAPE = 118.0880478165307%
Dy160: r2 = 0.7120213017101997 MAPE = 24300.252719057797%
Dy161: r2 = 0.663993084246444 MAPE = 50020.46690032092%
Dy162: r2 = 0.9589506557320704 MAPE = 19049.296841997755%
Dy163: r2 = 0.6334623406015654 MAPE = 5837.247119647717%
Dy164: r2 = 0.9196843973157196 MAPE = 29200.130852574148%
Dy165: r2 = 0.7560157969842775 MAPE = 482.66285397401737%
Dy165_m1: r2 = 0.7789776815983878 MAPE = 126.96995731879252%
Dy166: r2 = 0.8393657155704883 MAPE = 344.9232995238543%
Dy167: r2 = 0.5079551754583076 MAPE = 64.6007033168981%
Dy168: r2 = 0.7992653894048216 MAPE = 32.6920093622901%
Dy169: r2 = 0.9602114833675208 MAPE = 1.1235687059255242%
Dy170: r2 = 0.8508585019476713 MAPE = 0.4669616686401844%
Dy171: r2 = 0.5295633106519948 MAPE = 0.029361032824780135%
Dy172: r2 = 0.8606888908474195 MAPE = 0.004887226140693826%
Ho159: r2 = 0.852

In110_m1: r2 = -193.56036122430538 MAPE = 5.297702977843756e-15%
Sn110: r2 = -26.724108006065904 MAPE = 9.871846334973188e-15%
Xe122: r2 = -2.704183356787678 MAPE = 3.0989751594610493e-22%
Sm142: r2 = -98.29089418800423 MAPE = 2.799108901467386e-19%
Tc94: r2 = -83452267.20224355 MAPE = 1.5977829601440117e-20%
La134: r2 = -12438376.922209421 MAPE = 7.252827678021129e-21%
Rn216: r2 = -39.07996353626834 MAPE = 1.8981239611264472e-25%


In [305]:
with open(f"ml_models/depletion_model.pkl", 'wb') as f:
    model_data = pickle.dumps(depletion_model)
    f.write(model_data)